Up-to-date version: https://colab.research.google.com/drive/1CEQXZz8Ib5fH9X2PMFKpDLhGy6Immftl

## Mutual Information

Mutual information between two random variables $X$ and $Y$ defines how well $X$ explains $Y$ and vice versa.

$$I(X; Y) = H(X) - H(X|Y) = H(Y) - H(Y|X) \tag{1}$$

where $H(X)$ is the entropy of $X$ and $H(X|Y)$ is the conditional entropy of $X$ given $Y$. As it follows from $(1)$, mutual information is symmetric: $I(X; Y) = I(Y; X)$.

$ H(X) = \sum_{x} p(x) \log_2 {1 \over p(x)} \\ H(X|Y) = \sum_y p(y)H(X|Y=y) = \sum_y p(y) \sum_x p(x|y)log_2{1 \over p(x|y)}$

![](https://upload.wikimedia.org/wikipedia/commons/thumb/d/d4/Entropy-mutual-information-relative-entropy-relation-diagram.svg/1200px-Entropy-mutual-information-relative-entropy-relation-diagram.svg.png)

In the first task we'll examine the mutual information between

* independent variables $X$ and $Y$
* when $Y$ is a deterministic function of $X$
* when $Y$ is a deterministic function of $X$ plus some noise

Let's start simple. What's the information between $X$ and $Y$ when

```
X = [5, 5, 8]
Y = [3, 7, 7]
```

In [0]:
import numpy as np
from collections import namedtuple


def discrete_entropy(digits) -> float:
    """
    :param digits: realizations of a discrete random variable X
    :return estimated entropy of X in bits
    """
    unique, counts = np.unique(digits, return_counts=True)
    proba = counts / len(digits)
    entropy = sum(proba * np.log2(1 / proba))
    return entropy


def discrete_mutual_information(x, y) -> float:
    """
    :param x: realizations of a discrete random variable X
    :param y: realizations of a discrete random variable Y
    :return: estimated mutual information between X and Y in bits
    """
    # your code goes here
    # I(X; Y) = H(X) - H(X|Y)
    entropy_x = discrete_entropy(x)
    entropy_x_given_y = 0
    for y_unique in set(y):
        mask = y == y_unique
        x_given_y = x[mask]
        entropy_x_given_y += discrete_entropy(x_given_y) * np.mean(mask)
    mutual_information = entropy_x - entropy_x_given_y
    return mutual_information


xs = np.array([5, 5, 8])
ys = np.array([3, 7, 7])
print(f"Mutual information between X={xs} and Y={ys} is {discrete_mutual_information(xs, ys):.3f} bits")

Mutual information between X=[5 5 8] and Y=[3 7 7] is 0.252 bits


In [0]:
from collections import defaultdict


def test(size=1000, y_func=lambda x: x**2):
    information = defaultdict(list)
    y_entropy = defaultdict(list)
    x_entropy = []
    for trial in range(10):
        x = np.random.randint(low=0, high=10, size=size)

        y_random = np.random.randint(low=53, high=53 + 5, size=size)
        y_deterministic = y_func(x)
        noise = np.random.randint(low=0, high=10, size=size)
        y_noisy = y_deterministic + noise

        information['random'].append(discrete_mutual_information(x, y_random))
        information['deterministic'].append(discrete_mutual_information(x, y_deterministic))
        information['noisy'].append(discrete_mutual_information(x, y_noisy))

        x_entropy.append(discrete_entropy(x))
        y_entropy['random'].append(discrete_entropy(y_random))
        y_entropy['deterministic'].append(discrete_entropy(y_deterministic))
        y_entropy['noisy'].append(discrete_entropy(y_noisy))
    x_entropy = np.mean(x_entropy)
    for experiment_name in information.keys():
        max_information = min(x_entropy, np.mean(y_entropy[experiment_name]))
        print(f"{experiment_name}: I(X; Y) = {np.mean(information[experiment_name]):.4f} "
              f"± {np.std(information[experiment_name]):.4f} (maximum possible {max_information:.4f})")

test()

random: I(X; Y) = 0.0275 ± 0.0054 (maximum possible 2.3194)
deterministic: I(X; Y) = 3.3157 ± 0.0023 (maximum possible 3.3157)
noisy: I(X; Y) = 2.7564 ± 0.0294 (maximum possible 3.3157)


One of the greatest advantages of using mutual information is that it captures nonlinear interactions, and it does not require assumptions about the structure of the underlying data (i.e., it is model independent). You can try different `y_func` in the test above, but make sure both $X$'s and $Y$'s are discrete. Otherwise, you have to cluster continuous data in bins (subject of 4th lesson).
